<a href="https://colab.research.google.com/github/AmiraKnani/IA/blob/main/seance_numpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numpy.random import standard_normal, normal, multivariate_normal
from scipy.spatial.distance import euclidean

In [ ]:
# les juniors
mu1=np.array([1000,28])
sigma1=np.array([[5,0],[0,10]])
juniors=multivariate_normal(mean=mu1,cov=sigma1,size=500)


In [ ]:
# les seniors
mu2=np.array([5000,55])
sigma2=np.array([[100,0],[0,2]])
seniors=multivariate_normal(mean=mu2,cov=sigma2,size=50)

**Travail demandé:**

On veut decider pour un nouvel employé (requete) s'il appartient à la classe des employés seniors ou à la classe des employés jeniors ?
  
      Exemple: 
        
        Nom: Safa
        Salaire: 4500
        Age: 46

        Décision: Safa est senior

**Solution 1 (kmeans)**

1. Calculer la moyenne arithmétique de chaque classe (jeniors et seniors): mu_seniors et mu_juniors
2. Calculer la distance euclidienne entre :
     - requete et mu_seniors
     - requete et mu_juniors
3. Decider en se basant sur les distances obtenues

In [ ]:
#Clacul de la moyenne arithmétique des juniors
mu_juniors=sum(juniors) / len(juniors)
mu_juniors

array([999.87940335,  28.0149959 ])

In [ ]:
#Calcul de la moyenne arithmétique des seniors
mu_seniors=sum(seniors) / len(seniors)
mu_seniors

array([4999.98441971,   54.99090565])

In [ ]:
#Calcule de la distance euclidienne entre requete et mu_juniors
requete=np.array([4500,46])
dist_juniors=euclidean(mu_juniors,requete)
dist_juniors

3500.166803380019

In [ ]:
#Calcule de la distance euclidienne entre requete et mu_seniors
dist_seniors=euclidean(mu_seniors,requete)
dist_seniors

500.065252081535

In [ ]:
#Decider en se basant sur les distances obtenues
if (dist_juniors < dist_seniors):
  print("employé appartient à la classe des junior")
else :
  print("employé appartient à la classe des senior")


employé appartient à la classe des senior


**Solution 2 (knn)**

1. Concaténer verticalement les employés seniors et juniors dans une seule matrice que vous appelez X
  
        Note: 
        Utiliser la fonction X=np.vstack([X1,X2])

2. Génerer un vecteur labels de dimension=nombre total des employés 

  Ce vecteur labels contient des valeurs soit 1 ou 2 seulement comme suit :
    - valeur 1 qui indique que l'employé est junior
    - valeur 2 qui indique que l'employé est senior

3. Calculer les distances entre requete et tous les employés X

4. Trier les indices des employés par proximité à la requete (en se basant sur les distances obtenues).

  Garder juste les k=5 les plus proches seulement

5. En utilisant les k indices triés et les labels, determiner la classe (junior ou senior) qui est la plus proche de la requete ?

In [ ]:
#Concaténer verticalement les employés seniors et juniors dans une seule matrice que vous appelez X
X=np.vstack([juniors,seniors])

In [ ]:
#Initialiser le vecteur label
vec_lab=np.empty(len(X))

In [ ]:
#Remplir le vecteur label
for i in range(len(X)):
  if (X[i] in juniors) :
    vec_lab[i]=1
  else:
    vec_lab[i]=2

In [ ]:
#Initialiser le vecteur distance
vec_dist=np.empty(len(X))

In [ ]:
#Remplir le vecteur distance
for i in range(len(X)):
  vec_dist[i]=euclidean(X[i],requete)

In [ ]:
import pandas as pd

In [ ]:
#Concaténer les deux vecteurs label et distance dans dataframe
df=pd.DataFrame({'label': vec_lab,'dist': vec_dist})

In [ ]:
#Trier les distances par ordre croissant
df=df.sort_values(by="dist")

In [ ]:
#Exrtraire la nature des plus proches indices
j=0
s=0
for i in range(5):
  if df['label'].iloc[i]==1.0:
    j=j+1
  else: s=s+1

In [ ]:
##Decider en se basant sur les calculs précédents
if j>s :
  print("employé appartient à la classe des junior")
else: 
  print("employé appartient à la classe des senior")

employé appartient à la classe des senior


**Solution 3 (gmm):**

1. Estimer la loi gaussienne (mu,sigma) de chacune de 2 classes de population (seniors et juniors :
 - G_seniors : mu_seniors et sigma_seniors des seniors
 - G_juniors : mu_geniors et sigma_geniors des juniors

  Note: Utiliser les fonctions suivantes:
 - np.mean(X) pour calculer la moyenne arithmétique
 - np.cov(X.T) pour calculer la covariance

2. Calculer la vraisemblance de la requete dans chaque classe

  Note: La vraisemblance d'une donnée selon une classe est la probabilité de cette donnée selon la loi qui caractérise la classe.
  
  - La vraisemblance de la requete selon la classe seniors :
 
    vrais_requete_seniors =  probabilité de requete selon la loi normale multivariée (mu_seniors , sigma_seniors)
  - La vraisemblance de la requete selon la classe juniors :
    vrais_requete_juniors =  probabilité de requete selon la loi normale multivariée (mu_juniors , sigma_juniors)

  Note: Utiliser la fonction pdf de la classe multivariate_normal de module scipy.stats comme suit :
          Exemple: Soit la loi normale multivariée G=(mu,sigma). 
          La probabilité de x selon la loi G est calculé comme suit :
          mn=multivariate_normal(mu,sigma)
          mn.pdf(x)

3. Calculer la probabilité prior de chaque classe:

  Note: La probabilité prior d'une classe est le poids de la classe dans la population globale.

 - proba_prior_seniors=cardinal_seniors/nombre total d'employés
 - proba_prior_juniors=cardinal_juniors/nombre total d'employés

4. Pour décider sur l'appartenance d'un employé à une classe, on peut utiliser le théorème de Bayes. 

  Note: Théorème de Bayes
  La probailité posterior = probabilité n va appliquer le théoreme de Bayes pour calculer la probabilté posterior de la requete étant donn é chaque classe comme suit :
- proba_post_requete_seniors = vrais_requete_seniors x proba_prior_seniors
- proba_post_requete_juniors = vrais_requete_juniors x proba_prior_juniors

  L'employé requete appartient à la classe qui donne la probabilté posterior la plus elevée


In [ ]:
from sklearn import mixture

In [ ]:
gmm = mixture.GaussianMixture(n_components=2, covariance_type='full').fit(X)

In [ ]:
gmm.means_

array([[ 999.87940335,   28.0149959 ],
       [4999.98441971,   54.99090565]])